In [1]:
from nilearn import datasets
from nilearn.maskers import NiftiLabelsMasker

dataset = datasets.fetch_atlas_harvard_oxford('cort-maxprob-thr25-2mm')
atlas_filename = dataset.maps
labels = dataset.labels

#print('Atlas ROIs are located in nifti image (4D) at: %s' %    atlas_filename)  # 4D data

In [2]:

masker = NiftiLabelsMasker(labels_img=atlas_filename, standardize=True,
                           memory='nilearn_cache', verbose=5)


In [3]:
from nilearn.connectome import ConnectivityMeasure
import numpy as np
from nilearn import plotting
time_sta = 0
time_end = 45
size_step = 5

def compute_fc_series(data,time_sta,time_end,size_step):
    time_series = masker.fit_transform(data)
    series_matrix=[]

    while time_end <= time_series.shape[0]:
        time_series_this_window = time_series[time_sta:time_end]
        correlation_measure = ConnectivityMeasure(kind='correlation')
        correlation_matrix = correlation_measure.fit_transform([time_series_this_window])[0]
        time_sta += size_step
        time_end += size_step
        series_matrix.append(correlation_matrix)
    series_matrix = np.asarray(series_matrix)
    return series_matrix




In [4]:
#import test_fmri
import os
import numpy as np
import matplotlib.pyplot as plt
import nibabel as nib # common way of importing nibabel

import numpy as np
import heartpy as hp
import math
import pandas as pd
from scipy import stats as st
import bioread
from scipy import ndimage
from os import path

Filepath = '/Users/dragon/Downloads/BREATHE'
fmripath = os.path.join(Filepath, 'fmri')
ecgpath = os.path.join(Filepath, 'ecg')

fs = 2000  # sampling rate of Biosignalplux
lowFreq = .1
highFreq = 100

def getFilterData(rawData):
    data = np.asarray(rawData)
    data = np.interp(data, (data.min(), data.max()), (0, +1024))
    #data = ((((data / np.power(2, 16)) - 0.5) * 3) / 1019) * 2000  # ecg manual pdf page 5
    #data = hp.filter_signal(data, [lowFreq, highFreq], fs, order=4, filtertype='bandpass', return_top=False)

    return data


def getHF(rawData):
    so=1-size_step/(time_end-time_sta)
    working_data, measures = hp.process_segmentwise(rawData, fs, segment_width=45, calc_freq=True, segment_overlap =so )
    lnHF_HRV = np.log(measures['hf'])
    rmssd=np.log(measures['rmssd'])
    return lnHF_HRV, rmssd



#print(ecgpath)
def listdirs(path):
    return [d for d in os.listdir(path) if os.path.isdir(os.path.join(path, d))]


def extract_average_series(ts,sliding_window_size):
    #ts is a numpy array
    seriesofts= np.array_split(ts, int(ts.shape[-1]/sliding_window_size),axis=-1)
    seriesofts= np.asarray(seriesofts[:-1])
    last_element=np.asarray(seriesofts[-1])
    last_mean=np.mean(last_element,-1)
    mean_series = np.mean(seriesofts,-1)


    return np.concatenate((mean_series, np.expand_dims(last_mean, axis=0)), axis=0)

def builddatset(path):
    d_list = listdirs(path)
    all_subject_fmri_features=[]
    all_subject_ecg_features=[]
    for d in d_list:
        tem_path = os.path.join(path, d)
        if os.path.exists(os.path.join(tem_path,'final.feat','filtered_func_data.nii.gz')):
            mri_file = os.path.join(tem_path,'final.feat','filtered_func_data.nii.gz')
            fmri_feature=compute_fc_series(mri_file,time_sta,time_end,size_step)
            print(d+' fmri data extracted successfully')
            print('check the size')
            print(fmri_feature.shape)
            ecg_path_tem=os.path.join(tem_path, 'ecg')
            if os.path.exists(ecg_path_tem):
                for file in os.listdir(ecg_path_tem):
                    if file.endswith(".acq"):
                        print(os.path.join(ecg_path_tem, file))
                        data0 = bioread.read_file(os.path.join(ecg_path_tem, file))
                        rawdata = data0.channels[0].data
                        hf, rmssd = getHF(rawdata)
                        print(d + ' ecg feature extracted successfully')
                        print('check the data')
                        ecg_feature=rmssd[:fmri_feature.shape[0]]




            all_subject_fmri_features.append(fmri_feature)
            for i in range(ecg_feature.size):
                if np.isnan(ecg_feature[i]):
                    ecg_feature[i]=(ecg_feature[i]+ecg_feature[i-1])/2
            #print()
            print(ecg_feature)
            print(ecg_feature.shape)
            all_subject_ecg_features.append(ecg_feature)


        else:
            print('no_pre_fmri'+d)
    all_subject_fmri_features=np.asarray(all_subject_fmri_features)
    all_subject_ecg_features = np.asarray(all_subject_ecg_features)
    return all_subject_fmri_features, all_subject_ecg_features



def builddatset_nonav(path):
    d_list = listdirs(path)
    all_subject_fmri_features=[]
    for d in d_list:
        tem_path = os.path.join(path, d)
        if os.path.exists(os.path.join(tem_path,'final.feat','filtered_func_data.nii.gz')):
            mri_file = os.path.join(tem_path,'final.feat','filtered_func_data.nii.gz')
            img = nib.load(mri_file)
            img_data = img.get_fdata()
            print(img_data.shape)
            fmri_feature = img_data
            print(d+' fmri data extracted successfully')
            print('check the size')
            print(fmri_feature.shape)
            all_subject_fmri_features.append(fmri_feature)



        else:
            print('no_pre_fmri'+d)
    all_subject_fmri_features=np.asarray(all_subject_fmri_features)
    return all_subject_fmri_features



In [5]:
if __name__ == "__main__":
    #nonav_fmri_features=builddatset_nonav(fmripath)
    #print(nonav_fmri_features)
    #np.save('nonav_fmri_features', nonav_fmri_features)
    all_subject_fmri_features, all_subject_ecg_features =builddatset(fmripath)
    print(all_subject_fmri_features.shape)
    print(all_subject_ecg_features.shape)
    np.save('all_subject_fmri_features_region', all_subject_fmri_features)
    np.save('all_subject_ecg_features_region', all_subject_ecg_features)



[NiftiLabelsMasker.fit_transform] loading data from Nifti1Image(
shape=(91, 109, 91),
affine=array([[   2.,    0.,    0.,  -90.],
       [   0.,    2.,    0., -126.],
       [   0.,    0.,    2.,  -72.],
       [   0.,    0.,    0.,    1.]])
)
Resampling labels
[Memory]1.7s, 0.0min    : Loading _filter_and_extract...
__________________________________filter_and_extract cache loaded - 0.0s, 0.0min
s116 fmri data extracted successfully
check the size
(52, 48, 48)
/Users/dragon/Downloads/BREATHE/fmri/s116/ecg/BREATHE_s116_T1 (rest).acq


/Users/dragon/opt/anaconda3/envs/py37/lib/python3.7/site-packages/scipy/interpolate/fitpack2.py:253: UserWarning: 
The maximal number of iterations maxit (set to 20 by the program)
allowed for finding a smoothing spline with fp=s has been reached: s
too small.
There is an approximation returned but the corresponding weighted sum
of squared residuals does not satisfy the condition abs(fp-s)/s < tol.
  warnings.warn(message)
/Users/dragon/opt/anaconda3/envs/py37/lib/python3.7/site-packages/heartpy/analysis.py:522: UserWarning: Short signal.

---------Warning:---------
too few peak-peak intervals for (reliable) frequency domain measure computation, frequency output measures are still computed but treat them with caution!

HF is usually computed over a minimum of 1 minute of good signal. LF is usually computed over a minimum of 2 minutes of good signal.VLF is usually computed over a minimum of 5 minutes of good signal.The LF/HF ratio is usually computed over minimum 24 hours, although an a

s116 ecg feature extracted successfully
check the data
[5.10737914 5.05488869 5.00678943 5.00678943 4.63082558 5.29760279
 5.41359289 5.62323166 5.48146674 2.8772774  5.19851992 5.53761629
 5.2550591  5.16357548 5.20577042 5.19691618 5.34570567 5.45906578
 5.47342617 5.40536026 2.14608515 4.37092779 4.97431769 5.2182026
 5.22578332 5.18754914 5.51667382 5.23582646 5.31790696 5.35317859
 5.32305092 5.27011691 5.31291731 5.29963958 5.30751579 5.37208318
 5.3367458  5.32930697 5.30801746 5.32071475 5.30124651 5.3261009
 5.28368176 5.31067154 5.25645273 5.18939114 5.23072526 5.11213537
 5.40708671 5.37204856 5.33114089 5.13321609]
(52,)
[NiftiLabelsMasker.fit_transform] loading data from Nifti1Image(
shape=(91, 109, 91),
affine=array([[   2.,    0.,    0.,  -90.],
       [   0.,    2.,    0., -126.],
       [   0.,    0.,    2.,  -72.],
       [   0.,    0.,    0.,    1.]])
)
Resampling labels
[Memory]11.4s, 0.2min   : Loading _filter_and_extract...
__________________________________filter

/Users/dragon/opt/anaconda3/envs/py37/lib/python3.7/site-packages/heartpy/analysis.py:522: UserWarning: Short signal.

---------Warning:---------
too few peak-peak intervals for (reliable) frequency domain measure computation, frequency output measures are still computed but treat them with caution!

HF is usually computed over a minimum of 1 minute of good signal. LF is usually computed over a minimum of 2 minutes of good signal.VLF is usually computed over a minimum of 5 minutes of good signal.The LF/HF ratio is usually computed over minimum 24 hours, although an absolute minimum of 5 min has also been suggested.

For more info see: 
Shaffer, F., Ginsberg, J.P. (2017), An Overview of Heart Rate Variability Metrics and Norms.

Task Force of Pacing and Electrophysiology (1996), Heart Rate Variability, in: European Heart Journal, vol.17, issue 3, pp354-381

This warning will not repeat
  warnings.warn(msg, UserWarning)
/Users/dragon/opt/anaconda3/envs/py37/lib/python3.7/site-packages/sc

s118 ecg feature extracted successfully
check the data
[5.59638257 5.40203588 5.40203588 5.36031146 5.35258647 5.42056716
 5.30959139 5.65148487 5.49991431 5.61455856 5.61455856 5.59457113
 5.63628441 5.59910118 5.61362128 5.55165859 5.55929781 5.60480572
 5.59791416 5.52305587 5.45171924 5.38505204 5.42497719 5.48743326
 5.48500565 5.47437082 5.63010034 5.71069436 5.67641463 5.67641463
 5.67641463 5.4773272  5.47028587 5.45121237 5.63229918 5.53246021
 5.58771135 5.6203943  5.56459426 5.58889825 5.42309255 5.30453969
 5.20526853 5.76167184 5.76778256 5.76294709 5.57957294 5.83448981
 5.82687377 5.60896239 5.60896239 5.54158029]
(52,)
[NiftiLabelsMasker.fit_transform] loading data from Nifti1Image(
shape=(91, 109, 91),
affine=array([[   2.,    0.,    0.,  -90.],
       [   0.,    2.,    0., -126.],
       [   0.,    0.,    2.,  -72.],
       [   0.,    0.,    0.,    1.]])
)
Resampling labels
[Memory]27.2s, 0.5min   : Loading _filter_and_extract...
__________________________________filt

...

/Users/dragon/opt/anaconda3/envs/py37/lib/python3.7/site-packages/heartpy/analysis.py:522: UserWarning: Short signal.

---------Warning:---------
too few peak-peak intervals for (reliable) frequency domain measure computation, frequency output measures are still computed but treat them with caution!

HF is usually computed over a minimum of 1 minute of good signal. LF is usually computed over a minimum of 2 minutes of good signal.VLF is usually computed over a minimum of 5 minutes of good signal.The LF/HF ratio is usually computed over minimum 24 hours, although an absolute minimum of 5 min has also been suggested.

For more info see: 
Shaffer, F., Ginsberg, J.P. (2017), An Overview of Heart Rate Variability Metrics and Norms.

Task Force of Pacing and Electrophysiology (1996), Heart Rate Variability, in: European Heart Journal, vol.17, issue 3, pp354-381

This warning will not repeat
  warnings.warn(msg, UserWarning)
/Users/dragon/opt/anaconda3/envs/py37/lib/python3.7/site-packages/sc

s144 ecg feature extracted successfully
check the data
[4.56194925 4.61731773 4.44697441 4.64113558 4.64113558 5.05759242
 5.63612884 5.62747386 5.62359088 5.59760345 5.63602434 5.62092738
 5.7317142  5.72456635 5.72051551 5.72685008 5.76528022 5.75549089
 5.72221615 5.75389046 5.73833638 5.75398841 5.75004584 5.7679348
 5.74777401 5.70155968 5.6966116  5.74231462 5.72699096 5.67972265
 5.74773933 5.73764682 5.60479934 5.63821997 5.76746126 5.63837853
 5.72000257 5.75683296 5.71693295 5.72135258 5.73718437 5.73653209
 5.76397058 5.74068918 5.72758341 5.76901955 5.769236   5.78187444
 5.79316954 5.79411604 5.5714126  5.62608782]
(52,)
[NiftiLabelsMasker.fit_transform] loading data from Nifti1Image(
shape=(91, 109, 91),
affine=array([[   2.,    0.,    0.,  -90.],
       [   0.,    2.,    0., -126.],
       [   0.,    0.,    2.,  -72.],
       [   0.,    0.,    0.,    1.]])
)
Resampling labels
________________________________________________________________________________
[Memory] Callin

/Users/dragon/opt/anaconda3/envs/py37/lib/python3.7/site-packages/heartpy/analysis.py:522: UserWarning: Short signal.

---------Warning:---------
too few peak-peak intervals for (reliable) frequency domain measure computation, frequency output measures are still computed but treat them with caution!

HF is usually computed over a minimum of 1 minute of good signal. LF is usually computed over a minimum of 2 minutes of good signal.VLF is usually computed over a minimum of 5 minutes of good signal.The LF/HF ratio is usually computed over minimum 24 hours, although an absolute minimum of 5 min has also been suggested.

For more info see: 
Shaffer, F., Ginsberg, J.P. (2017), An Overview of Heart Rate Variability Metrics and Norms.

Task Force of Pacing and Electrophysiology (1996), Heart Rate Variability, in: European Heart Journal, vol.17, issue 3, pp354-381

This warning will not repeat
  warnings.warn(msg, UserWarning)
/Users/dragon/opt/anaconda3/envs/py37/lib/python3.7/site-packages/sc

s210 ecg feature extracted successfully
check the data


IndexError: index 52 is out of bounds for axis 0 with size 52

In [ ]:
# Plot the correlation matrix

# Make a large figure
# Mask the main diagonal for visualization:
np.fill_diagonal(correlation_matrix, 0)
# The labels we have start with the background (0), hence we skip the
# first label
# matrices are ordered for block-like representation
plotting.plot_matrix(correlation_matrix, figure=(10, 8), labels=labels[1:],
                     vmax=0.8, vmin=-0.8,
                     reorder=True)